<center><img src="https://media-exp1.licdn.com/dms/image/C4E22AQEbIXZiRVkJPQ/feedshare-shrink_2048_1536/0?e=1605139200&v=beta&t=opmJLG_veDflQesF9uJvIhSXbmwPY3DGSlbZo1v-w2k" width="1000"></center>

# Programa de Especialización en Python

## Tema 8. Análisis de Discriminante Lineal: Caso Practico

### Prof. Manuel Sigüeñas, M.Sc.(c)

In [1]:
import pandas as pd
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score

%matplotlib inline

### Cargar los datos

Porcentajes diarios de rendimiento para el índice bursátil S&P 500 entre 2001 y 2005.

- **Year**: el año en que se registró la observación.
- **Lag1**: porcentaje de devolución del día anterior
- **Lag2**: porcentaje de devolución de 2 días anteriores
- **Lag3**: Porcentaje de devolución de 3 días anteriores
- **Lag4**: porcentaje de devolución de 4 días anteriores
- **Lag5**: porcentaje de devolución de 5 días anteriores
- **Volume**: volumen de acciones negociadas (número de acciones diarias negociadas en miles de millones)
- **Today**: porcentaje de retorno para hoy
- **Direction**: un factor con niveles hacia abajo y hacia arriba que indica si el mercado tuvo un rendimiento positivo o negativo en un día determinado

En Python, podemos ajustar un modelo LDA usando la función LinearDiscriminantAnalysis(), que es parte del módulo discriminant_analysis de la librería de ML sklearn. Ajustaremos el modelo utilizando solo las observaciones anteriores a 2005, y luego probaremos el modelo con los datos de 2005.

In [2]:
df = pd.read_csv('D:/Python/2. Nivel II/8/datos/Smarket.csv', usecols=range(1,10), index_col=0, parse_dates=True)
df.head()

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001-01-01,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001-01-01,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001-01-01,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001-01-01,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001-01-01,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


## LDA

In [3]:
X_train = df[:'2004'][['Lag1','Lag2']]
y_train = df[:'2004']['Direction']
y_train2 = df[:'2004']['Direction']
X_test = df['2005':][['Lag1','Lag2']]
y_test = df['2005':]['Direction']

In [5]:
lda = LinearDiscriminantAnalysis()
model = lda.fit(X_train,# los valores de los predictores de entrenamiento
                y_train)#los valores de y de entrenamiento

print(model.priors_)

[0.49198397 0.50801603]


La salida LDA indica probabilidades previas de $\hat{π}_1=0.492$ y $\hat{π}_2=0.508$; n otras palabras, el 49.2% de las observaciones de entrenamiento corresponden a días durante los cuales el mercado Down.

In [6]:
print(model.means_)

[[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]


Lo anterior proporciona los medios grupales; estos son el promedio de cada predictor dentro de cada clase, y son utilizados por LDA como estimaciones de  $μ_k$. Esto sugiere que existe una tendencia a que los retornos de los 2 días anteriores sean negativos en los días en que el mercado aumenta, y una tendencia a que los retornos de los días anteriores sean positivos en los días en que el mercado declina.

In [7]:
print(model.coef_)

[[-0.05544078 -0.0443452 ]]


Los coeficientes de salida discriminantes lineales proporcionan la combinación lineal de Lag1y Lag2que se utilizan para formar la regla de decisión LDA.

$If −0.0554×Lag1−0.0443×Lag2$ es grande, entonces el clasificador LDA predecirá un aumento del mercado, y si es pequeño, entonces el clasificador LDA pronosticará una disminución del mercado. 



In [8]:
pred=model.predict(X_test)
print(np.unique(pred, 
                return_counts=True))

(array(['Down', 'Up'], dtype='<U4'), array([ 70, 182], dtype=int64))


El modelo asignó 70 observaciones a la clase "Down" y 182 observaciones a la clase "Up". Veamos la matriz de confusión para ver cómo le está yendo a este modelo. Queremos comparar la clase predicha (que podemos encontrar en pred) con la clase verdadera (que se encuentra en $y_test$)

### Matriz de confusión datos de entrenamiento

In [9]:
pred2=model.predict(X_train)

In [10]:
print(np.unique(pred2, return_counts=True))

(array(['Down', 'Up'], dtype='<U4'), array([328, 670], dtype=int64))


In [11]:
print(confusion_matrix(pred2, y_train2))

[[168 160]
 [323 347]]


In [12]:
print(classification_report(y_train2, #los valores predichos
                            pred2, #los valores reales
                            digits=2)) #digitos

              precision    recall  f1-score   support

        Down       0.51      0.34      0.41       491
          Up       0.52      0.68      0.59       507

    accuracy                           0.52       998
   macro avg       0.52      0.51      0.50       998
weighted avg       0.52      0.52      0.50       998



### Matriz de confusión datos de prueba

In [13]:
print(confusion_matrix(pred, y_test))

[[ 35  35]
 [ 76 106]]


In [15]:
confusion_matrix = pd.crosstab(pred, y_test)
confusion_matrix

Direction,Down,Up
row_0,,
Down,35,35
Up,76,106


In [16]:
print(classification_report(y_test, pred, digits=2))

              precision    recall  f1-score   support

        Down       0.50      0.32      0.39       111
          Up       0.58      0.75      0.66       141

    accuracy                           0.56       252
   macro avg       0.54      0.53      0.52       252
weighted avg       0.55      0.56      0.54       252



## QDA

In [15]:
qda = QuadraticDiscriminantAnalysis()

In [16]:

model2 = qda.fit(X_train, #X DE ENTRENAMIENTO 
                 y_train) #Y DE ENTRENAMIENTO
print(model2.priors_)

[0.49198397 0.50801603]


In [17]:
print(model2.means_)

[[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]


### Matriz de confusión datos de entrenamiento

In [21]:
pred4=model2.predict(X_train)
print(np.unique(pred4, return_counts=True))

(array(['Down', 'Up'], dtype=object), array([318, 680], dtype=int64))


In [20]:
print(classification_report(y_train, pred4, digits=2))

              precision    recall  f1-score   support

        Down       0.51      0.33      0.40       491
          Up       0.52      0.69      0.59       507

    accuracy                           0.51       998
   macro avg       0.51      0.51      0.50       998
weighted avg       0.51      0.51      0.50       998



### Matriz de confusión datos de prueba

In [23]:
pred3=model2.predict(X_test)
print(np.unique(pred3, return_counts=True))

(array(['Down', 'Up'], dtype=object), array([ 50, 202], dtype=int64))


In [24]:
print(classification_report(y_test, pred3, digits=2))

              precision    recall  f1-score   support

        Down       0.60      0.27      0.37       111
          Up       0.60      0.86      0.71       141

    accuracy                           0.60       252
   macro avg       0.60      0.56      0.54       252
weighted avg       0.60      0.60      0.56       252

